# Goal is to display all workouts
#### * Query database
#### * Transform models into viewmodels
#### * Display data to user

In [30]:
import sqlite3
import json

In [13]:
DATABASE = 'workout.db'

### Define function to query database

In [14]:
def query_db(query, args=(), one=False):
    conn = sqlite3.connect(DATABASE)
    conn.row_factory = sqlite3.Row
    cur = conn.execute(query, args)
    rv = cur.fetchall()
    conn.commit()
    cur.close()
    return (rv[0] if rv else None) if one else rv

### Query all workouts & topsets completed by Gabe(PersonId=1)

In [22]:
topsets = query_db("""
    SELECT P.Name AS PersonName, W.WorkoutId, W.StartDate, E.Name AS ExerciseName, T.Repetitions, T.Weight
    FROM Person P
         LEFT JOIN Workout W ON P.PersonId=W.PersonId
         LEFT JOIN TopSet T ON W.WorkoutId=T.WorkoutId
         LEFT JOIN Excercise E ON T.ExcerciseId=E.ExcerciseId
    WHERE P.PersonId=?""", [1])

for topset in topsets:
    print(f'WorkoutId: {topset["WorkoutId"]}, Person:{topset["PersonName"]}, StartDate: {topset["StartDate"]}, Exercise: {topset["ExerciseName"]}, Repetitions: {topset["Repetitions"]}, Weight: {topset["Weight"]}kg')

WorkoutId: 1, Person:Gabe, StartDate: 2022-06-29 00:00:00.000, Exercise: Bench, Repetitions: 11, Weight: 40kg
WorkoutId: 2, Person:Gabe, StartDate: 2022-07-07 00:00:00.000, Exercise: Squat, Repetitions: 5, Weight: 65kg
WorkoutId: 2, Person:Gabe, StartDate: 2022-07-07 00:00:00.000, Exercise: Hotep, Repetitions: 6, Weight: 30kg
WorkoutId: 3, Person:Gabe, StartDate: 2022-07-12 00:00:00.000, Exercise: Bench, Repetitions: 4, Weight: 60kg
WorkoutId: 3, Person:Gabe, StartDate: 2022-07-12 00:00:00.000, Exercise: Deadlift, Repetitions: 9, Weight: 100kg


### Transform entity to view model with the following schema
<pre>
{  
    "name": "Gabe",  
    "workouts": [  
        {  
            "StartDate": "2022-06-29 00:00:00.000",  
            "TopSets": [  
                {  
                        "ExerciseName": "Squat",  
                        "Weight": 80,  
                        "Repetitions": 8  
                    },  
                    {  
                        "ExerciseName": "Bench",  
                        "Weight": 60,  
                        "Repetitions": 4  
                    }  
                ]  
        },  
        {  
            "StartDate": "2022-07-01 00:00:00.000",  
            "TopSets": [  
                {  
                    "ExerciseName": "Squat",  
                    "Weight": 85,  
                    "Repetitions": 4  
                },  
                {  
                    "ExerciseName": "Bench",  
                    "Weight": 65,  
                    "Repetitions": 5  
                }  
            ]  
        }  
    ]  
} 
</pre>

#### 1. Extract name from rows

In [37]:
def get_name(topsets):
    return topsets[0]['PersonName']

get_name(topsets)

'Gabe'

#### 2. Group rows into workouts using WorkoutId column

In [43]:
def get_workouts(topsets):
    # Get all unique workout_ids (No duplicates)
    workout_ids = set([t['WorkoutId'] for t in topsets])

    # Group topsets into workouts
    workouts = []
    for workout_id in workout_ids:
        topsets_in_workout = [t for t in topsets if t['WorkoutId'] == workout_id]
        workouts.append({
            'WorkoutId': workout_id,
            'StartDate': topsets_in_workout[0]['StartDate'],
            'Exercises': [{"ExerciseName": t['ExerciseName'], "Weight": t['Weight'], "Repetitions": t['Repetitions'] } for t in topsets_in_workout]
        })
    return workouts

In [46]:
print(json.dumps(get_workouts(topsets), indent=2))

[
  {
    "WorkoutId": 1,
    "StartDate": "2022-06-29 00:00:00.000",
    "Exercises": [
      {
        "ExerciseName": "Bench",
        "Weight": 40,
        "Repetitions": 11
      }
    ]
  },
  {
    "WorkoutId": 2,
    "StartDate": "2022-07-07 00:00:00.000",
    "Exercises": [
      {
        "ExerciseName": "Squat",
        "Weight": 65,
        "Repetitions": 5
      },
      {
        "ExerciseName": "Hotep",
        "Weight": 30,
        "Repetitions": 6
      }
    ]
  },
  {
    "WorkoutId": 3,
    "StartDate": "2022-07-12 00:00:00.000",
    "Exercises": [
      {
        "ExerciseName": "Bench",
        "Weight": 60,
        "Repetitions": 4
      },
      {
        "ExerciseName": "Deadlift",
        "Weight": 100,
        "Repetitions": 9
      }
    ]
  }
]


### 3. Compose functions together to acheive final model

In [47]:
def convert_to_view_model(topsets):
    return {
        'PersonName': get_name(topsets),
        'Workouts': get_workouts(topsets)
    }

In [48]:
print(json.dumps(convert_to_view_model(topsets), indent=2))

{
  "PersonName": "Gabe",
  "Workouts": [
    {
      "WorkoutId": 1,
      "StartDate": "2022-06-29 00:00:00.000",
      "Exercises": [
        {
          "ExerciseName": "Bench",
          "Weight": 40,
          "Repetitions": 11
        }
      ]
    },
    {
      "WorkoutId": 2,
      "StartDate": "2022-07-07 00:00:00.000",
      "Exercises": [
        {
          "ExerciseName": "Squat",
          "Weight": 65,
          "Repetitions": 5
        },
        {
          "ExerciseName": "Hotep",
          "Weight": 30,
          "Repetitions": 6
        }
      ]
    },
    {
      "WorkoutId": 3,
      "StartDate": "2022-07-12 00:00:00.000",
      "Exercises": [
        {
          "ExerciseName": "Bench",
          "Weight": 60,
          "Repetitions": 4
        },
        {
          "ExerciseName": "Deadlift",
          "Weight": 100,
          "Repetitions": 9
        }
      ]
    }
  ]
}
